In [0]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder.appName("DataQualityCheck").getOrCreate()



In [0]:
%sql
SELECT * FROM workspace.default.raw_data LIMIT 10;


In [0]:
%sql
SELECT 
    COUNT(*) AS total_records,
    SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END) AS missing_name,
    SUM(CASE WHEN speciality IS NULL THEN 1 ELSE 0 END) AS missing_speciality,
    SUM(CASE WHEN education IS NULL THEN 1 ELSE 0 END) AS missing_education,
    SUM(CASE WHEN experience IS NULL THEN 1 ELSE 0 END) AS missing_experience,
    SUM(CASE WHEN city IS NULL THEN 1 ELSE 0 END) AS missing_city,
    SUM(CASE WHEN state IS NULL THEN 1 ELSE 0 END) AS missing_state,
    SUM(CASE WHEN pincode IS NULL THEN 1 ELSE 0 END) AS missing_pincode
FROM workspace.default.raw_data;


In [0]:
%sql
SELECT name, speciality, education, experience, COUNT(*) AS count
FROM workspace.default.raw_data
GROUP BY name, speciality, education, experience
HAVING COUNT(*) > 1;


In [0]:
%sql
SELECT * FROM workspace.default.raw_data 
WHERE TRY_CAST(pincode AS INT) IS NULL AND pincode IS NOT NULL;


In [0]:
%sql
SELECT * FROM workspace.default.raw_data
WHERE email IS NULL OR email NOT LIKE '%@%.%';


In [0]:
%sql
UPDATE workspace.default.raw_data
SET speciality = 'Unknown'
WHERE speciality IS NULL;


In [0]:
%sql
SELECT name, COUNT(DISTINCT clinic_name) AS clinic_count
FROM workspace.default.raw_data
GROUP BY name
HAVING COUNT(DISTINCT clinic_name) > 1;


In [0]:
%sql
SELECT * FROM workspace.default.raw_data
WHERE phone IS NULL OR TRIM(phone) = '';


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.default.cleaned_data AS 
WITH ranked_data AS (
    SELECT 
        record_id, 
        url, 
        sheet_doc_name, 
        sheet_speciality, 
        name, 
        clinic_name, 
        education, 
        experience, 
        speciality, 
        address, 
        mci, 
        passing_year, 
        memberships, 
        fees, 
        timing, 
        awards, 
        specializations, 
        full_education, 
        full_experience, 
        services, 
        name_ratio, 
        speciality_ratio, 
        clinic_Locationn, 
        doctor_location, 
        `Languages spoken` AS languages_spoken, 
        clinic__name1, 
        address1, 
        fee1, 
        clinic__name2, 
        address2, 
        fee2, 
        clinic__name3, 
        address3, 
        fee3, 
        phone, 
        alternate_email, 
        email, 
        locality, 
        city, 
        pincode, 
        state, 
        clinic__name4, 
        address4, 
        fee4, 
        ROW_NUMBER() OVER (
            PARTITION BY name, speciality, education, experience 
            ORDER BY record_id
        ) AS row_num
    FROM workspace.default.raw_data
)
SELECT * FROM ranked_data WHERE row_num = 1;

In [0]:
%sql
select * from workspace.default.cleaned_data limit 10 ;

In [0]:
%sql
select 'cleaned',count (*) from workspace.default.cleaned_data 
union all 
select 'dedupe',count (*) from workspace.default.raw_data

In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET speciality = 'Unknown'
WHERE speciality IS NULL;




In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET education = 'Not Available'
WHERE education IS NULL;

In [0]:
%sql
DELETE FROM workspace.default.cleaned_data
WHERE email IS NOT NULL AND email NOT LIKE '%@%.%';

In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET phone = REGEXP_REPLACE(phone, '[^0-9]', '')
WHERE phone IS NOT NULL;


In [0]:
%sql
DELETE FROM workspace.default.cleaned_data
WHERE TRY_CAST(pincode AS INT) IS NULL AND pincode IS NOT NULL;


In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET phone = CASE 
                WHEN phone IS NULL OR TRIM(phone) = '' THEN 'Unknown' 
                ELSE phone 
            END,
    email = CASE 
                WHEN email IS NULL OR TRIM(email) = '' THEN 'Unknown' 
                ELSE email 
            END,
    alternate_email = CASE 
                         WHEN alternate_email IS NULL OR TRIM(alternate_email) = '' THEN 'Unknown' 
                         ELSE alternate_email 
                     END;


In [0]:
%sql
select count(*) from workspace.default.cleaned_data

In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET experience = LEFT(REGEXP_REPLACE(experience, '[^0-9]', ''), 2);






In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET name = TRIM(name),
    speciality = TRIM(speciality),
    clinic_name = TRIM(clinic_name),
    education = TRIM(education),
    experience = TRIM(experience),
    address = TRIM(address),
    city = TRIM(city),
    state = TRIM(state);


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.default.final_cleaned_data AS 
SELECT * FROM workspace.default.cleaned_data;


In [0]:
%sql
SELECT * FROM workspace.default.cleaned_data;

In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET phone = NULL
WHERE LENGTH(phone) < 10;


In [0]:
%sql
SELECT phone FROM workspace.default.cleaned_data WHERE LENGTH(phone) < 10;


In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET Name = TRIM(SPLIT(Name, ',')[0]);


In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET pincode = NULL
WHERE LENGTH(pincode) < 6;


In [0]:
%sql
ALTER TABLE workspace.default.cleaned_data ADD COLUMN college_name STRING;




In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET college_name = TRIM(try_element_at(SPLIT(passing_year, '-'), 2));

In [0]:
%sql
ALTER TABLE workspace.default.cleaned_data 
ADD COLUMN mci_number STRING;





In [0]:
%sql
ALTER TABLE workspace.default.cleaned_data 
 
ADD COLUMN state_medical_council STRING;


In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET mci_number = SPLIT(mci, ' ')[0],
    state_medical_council = CASE 
        WHEN SIZE(SPLIT(mci, ' ')) > 1 THEN SPLIT(mci, ' ', 2)[1] 
        ELSE NULL 
    END;


In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET 
    -- 1️⃣ Cleaning MCI Number (Ensures full retention of valid MCI numbers)
    mci = CASE 
        -- Handle null-like values
        WHEN LOWER(TRIM(mci)) IN ('ull,', 'null', '', 'mci') THEN NULL
        
        -- Handle standard Medical Council format (Extract registration number)
        WHEN mci RLIKE '.* Medical Council.*' THEN 
            REGEXP_REPLACE(mci, '^([A-Z/-]*\\d+(?:/\\d+/\\d+)?).*$', '$1')

        -- Handle prefixed registration numbers (A-, G-, etc.), ensuring prefix is preserved
        WHEN mci RLIKE '^[A-Z]-?\\d+' THEN 
            TRIM(mci)
            
        -- Handle KMC format (e.g., "KMC 12345" → "12345")
        WHEN mci RLIKE '^KMC\\s+\\d+' THEN 
            REGEXP_EXTRACT(mci, 'KMC\\s+(\\d+)')
            
        -- Handle pure numeric with possible slashes
        WHEN mci RLIKE '^\\d+(/\\d+/\\d+)?$' THEN 
            TRIM(mci)
            
        -- Remove all non-numeric characters except slashes for other cases
        ELSE REGEXP_REPLACE(TRIM(mci), '[^0-9/A-Z-]', '')
    END,

    -- 2️⃣ Extracting State Medical Council (Handles more variations)
    state_medical_council = CASE 
        -- Handle null-like values
        WHEN LOWER(TRIM(mci)) IN ('ull,', 'null', '', 'mci') THEN NULL
        
        -- Handle pure state names (normalization)
        WHEN mci RLIKE '^[A-Za-z\\s-]+$' THEN 
            CASE 
                WHEN LOWER(TRIM(mci)) IN ('odisha', 'orissa') THEN 'Orissa Medical Council'
                WHEN LOWER(TRIM(mci)) = 'kmc' THEN 'Karnataka Medical Council'
                ELSE CONCAT(TRIM(mci), ' Medical Council')
            END
        
        -- Handle KMC format
        WHEN mci RLIKE '^KMC\\s+\\d+' THEN 'Karnataka Medical Council'
        
        -- Handle state name with number (e.g., "Madhya Pradesh 7726")
        WHEN mci RLIKE '^[A-Za-z\\s]+\\d+' THEN 
            CONCAT(REGEXP_EXTRACT(mci, '^([A-Za-z\\s]+)'), ' Medical Council')

        -- Extract known state councils from structured Medical Council format
        WHEN mci RLIKE '.* Medical Council.*' THEN 
            REGEXP_EXTRACT(mci, '.*((?:Karnataka|Maharashtra|Delhi|Gujarat|Rajasthan|Tamil Nadu|Andhra Pradesh|West Bengal|Uttar Pradesh|Punjab|Madhya Pradesh|Telangana|Assam|Haryana|Orissa|Chhattisgarh|Kerala|Travancore-Cochin)\\s+(?:State\\s+)?(?:Medical|Dental|Homoeopathy|Ayurvedic|Unani)\\s+Council[^,]*)')
            
        -- Handle special cases like Rehabilitation Council or Dubai Medical Registry
        WHEN mci RLIKE '.*(Rehabilitation Council|Medical Registry).*' THEN 
            REGEXP_EXTRACT(mci, '.*((?:Rehabilitation Council of India|Dubai Medical Registry)[^,]*)')
            
        ELSE state_medical_council
    END;


In [0]:
%sql
select mci,state_medical_council from workspace.default.cleaned_data ;

In [0]:
%sql
ALTER TABLE workspace.default.cleaned_data 
ADD COLUMNS (speciality1 STRING, speciality2 STRING);


In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET 
    -- Standardizing & extracting first specialty
    speciality1 = CASE 
        WHEN LOWER(TRIM(speciality)) IN ('na', 'unknown', '') THEN NULL
        WHEN speciality RLIKE '(?i)\\b(and|&|AND|-)\\b' THEN INITCAP(REGEXP_EXTRACT(speciality, '^(.*?)(?i)\\b(and|&|AND|-)\\b.*$', 1))
        WHEN speciality LIKE '%Gynecologist%' OR speciality LIKE '%Gynaecologist%' THEN 'Gynaecology'
        WHEN speciality LIKE '%Obstetrician%' OR speciality LIKE '%Obstetrics%' THEN 'Obstetrics'
        ELSE INITCAP(TRIM(speciality))  -- Capitalize first letter of each word
    END,

    -- Standardizing & extracting second specialty
    speciality2 = CASE 
        WHEN LOWER(TRIM(speciality)) IN ('na', 'unknown', '') THEN NULL
        WHEN speciality RLIKE '(?i)\\b(and|&|AND|-)\\b' THEN INITCAP(REGEXP_EXTRACT(speciality, '^(?:.*?)(?i)\\b(and|&|AND|-)\\b(.*)$', 2))
        WHEN speciality LIKE '%Gynecologist%' OR speciality LIKE '%Gynaecologist%' THEN 'Gynaecology'
        WHEN speciality LIKE '%Obstetrician%' OR speciality LIKE '%Obstetrics%' THEN 'Obstetrics'
        ELSE NULL  -- If only one specialty exists, keep speciality2 NULL
    END;


In [0]:
%sql
select speciality1,speciality2 from workspace.default.cleaned_data;

In [0]:
%sql
SELECT city, COUNT(*) as doctor_count
FROM workspace.default.cleaned_data
GROUP BY city
ORDER BY doctor_count DESC;


In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET locality = INITCAP(locality),  -- Convert locality to Camel Case
    city = INITCAP(
              CASE 
                  WHEN locality LIKE '%,%' 
                  THEN SPLIT(locality, ',')[0]  -- Extract text before the first comma
                  ELSE SPLIT(locality, ' ')[0]  -- Extract the first word if no comma exists
              END
           );




In [0]:
%sql
UPDATE workspace.default.cleaned_data
SET 
    college_name = TRIM(REGEXP_REPLACE(passing_year, ',?\\s*\\d{4}$', '')),  
    passing_year = TRIM(REGEXP_EXTRACT(passing_year, '(\\d{4})$'));


In [0]:
%sql
select * from workspace.default.cleaned_data

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load data from Databricks table
df = spark.sql("""
    SELECT city, speciality, COUNT(*) as doctor_count
    FROM workspace.default.cleaned_data
    GROUP BY city, speciality
""").toPandas()

# Pivot for Heatmap
df_pivot = df.pivot(index="city", columns="speciality", values="doctor_count")

# Plot Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df_pivot, cmap="coolwarm", annot=True, fmt=".0f", linewidths=0.5)

plt.title("Doctor Distribution by City and Specialization")
plt.xlabel("Specialization")
plt.ylabel("City")
plt.xticks(rotation=45)
plt.show()


In [0]:
df = spark.sql("""
    SELECT city, COUNT(*) as doctor_count
    FROM workspace.default.cleaned_data
    GROUP BY city
""").toPandas()

plt.figure(figsize=(12, 6))
sns.barplot(data=df, x="city", y="doctor_count", palette="viridis")
plt.xticks(rotation=45)
plt.xlabel("City")
plt.ylabel("Number of Doctors")
plt.title("Distribution of Doctors by City")
plt.show()


In [0]:
df = spark.sql("""
    SELECT city, speciality, COUNT(*) as doctor_count
    FROM workspace.default.cleaned_data
    GROUP BY city, speciality
""").toPandas()

df_pivot = df.pivot(index="city", columns="speciality", values="doctor_count")

plt.figure(figsize=(12, 8))
sns.heatmap(df_pivot, cmap="coolwarm", annot=True, fmt=".0f", linewidths=0.5)
plt.title("Doctor Distribution by City and Specialization")
plt.xlabel("Specialization")
plt.ylabel("City")
plt.xticks(rotation=45)
plt.show()


In [0]:
df = spark.sql("""
    SELECT speciality, COUNT(*) as doctor_count
    FROM workspace.default.cleaned_data
    GROUP BY speciality
""").toPandas()

plt.figure(figsize=(10, 10))
plt.pie(df["doctor_count"], labels=df["speciality"], autopct="%1.1f%%", colors=sns.color_palette("pastel"))
plt.title("Distribution of Doctors by Specialization")
plt.show()


In [0]:
df = spark.sql("""
    SELECT state, COUNT(*) as doctor_count
    FROM workspace.default.cleaned_data
    GROUP BY state
""").toPandas()

plt.figure(figsize=(10, 10))
plt.pie(df["doctor_count"], labels=df["state"], autopct="%1.1f%%", colors=sns.color_palette("coolwarm"))
plt.title("Distribution of Doctors by State")
plt.show()


In [0]:
%sql
select experience from workspace.default.cleaned_data

In [0]:
%sql
select count(*) ,speciality, experience from workspace.default.cleaned_data group by 2,3

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load data from Databricks table
df = spark.sql("""
    SELECT COUNT(*) as doctor_count, speciality, experience
    FROM workspace.default.cleaned_data
    GROUP BY speciality, experience
""").toPandas()

# Plot bar chart
plt.figure(figsize=(14, 8))
sns.barplot(data=df, x="experience", y="doctor_count", hue="speciality", dodge=True)

plt.title("Doctor Count by Speciality and Experience")
plt.xlabel("Years of Experience")
plt.ylabel("Number of Doctors")
plt.xticks(rotation=45)
plt.legend(title="Speciality")
plt.show()


In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load data from Databricks table
df = spark.sql("""
    SELECT COUNT(*) as doctor_count, speciality, experience
    FROM workspace.default.cleaned_data
    WHERE experience IS NOT NULL AND speciality IS NOT NULL
    GROUP BY speciality, experience
    HAVING COUNT(*) > 0
""").toPandas()

# Pivot for Heatmap
df_pivot = df.pivot(index="experience", columns="speciality", values="doctor_count")

# Plot Heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(df_pivot, cmap="coolwarm", annot=True, fmt=".0f", linewidths=0.5)

plt.title("Doctor Count by Speciality and Experience")
plt.xlabel("Speciality")
plt.ylabel("Years of Experience")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


In [0]:
%python
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load data from Databricks table
df = spark.sql("""
    SELECT city, speciality, experience, COUNT(*) as doctor_count
    FROM workspace.default.cleaned_data
    WHERE city IS NOT NULL AND speciality IS NOT NULL AND experience IS NOT NULL
    GROUP BY city, speciality, experience
    HAVING COUNT(*) > 0
""").toPandas()

# Pivot for Heatmap
df_pivot = df.pivot_table(index=["city", "experience"], columns="speciality", values="doctor_count", aggfunc="sum")

# Check if the pivot table is empty
if not df_pivot.empty:
    # Plot Heatmap
    plt.figure(figsize=(16, 10))
    sns.heatmap(df_pivot, cmap="coolwarm", annot=True, fmt=".0f", linewidths=0.5)

    plt.title("Doctor Count by City, Speciality, and Experience")
    plt.xlabel("Speciality")
    plt.ylabel("City & Experience")
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.show()
else:
    print("No data available to plot the heatmap.")

In [0]:
%sql
 SELECT *
    FROM workspace.default.cleaned_data ;
    

In [0]:
%sql
describe table workspace.default.cleaned_data ;

In [0]:
%sql
SELECT 
    record_id,
    Name,
    url,
    --Gender,
    full_experience,
    speciality1 AS `Specialty 1`,
    speciality2 AS `Specialty 2`,
    education AS Qualification,
    clinic_name AS `Clinic name`,
    address AS `Street name`,
    City,
    State,
    pincode as Pin,
    college_name AS `College name`,
    passing_year AS `Year of graduation`,
    mci AS `Mci No`,
    state_medical_council AS `State Medical Council`
FROM workspace.default.cleaned_data;
